In [ ]:
import numpy as np
import cv2 as cv
%matplotlib inline
import matplotlib.pyplot as plt
def interrupt_function():
    while(True):
        k = cv.waitKey(33)
        if k == -1:  # if no key was pressed, -1 is returned
            continue
        else:
            break
img = cv.imread(r"c://png.png")
plt.imshow(img)
plt.show(block=False)
img_colorless = cv.cvtColor(img,cv.COLOR_RGB2GRAY)
plt.imshow(img_colorless,cmap='gray')
plt.show(block=False)
img_binarised = cv.threshold(img_colorless,127,255,cv.THRESH_BINARY)[1]
plt.imshow(img_binarised,cmap='gray')
imgqwe = cv.connectedComponents(img_binarised)
q = 
plt.show()
